# Nonlinear Regression (Anton Maliev)

# k-Nearest Neighbors (Lester Fan)

# Lasso Least Squares Regression (Bill Lee)

### Bill Lee, 165003682 Lasso Least Squares


In [1]:
def rrLS(y, X, lammy):
    # adding ones to the bottom 
    y = np.reshape(y, (X.shape[0],1))
    eX = np.concatenate((X,np.ones((X.shape[0],1))),axis=1)
    extex = np.matmul(eX.T,eX)
    lammyI = lammy*np.identity(extex.shape[0])
    extexinv = np.linalg.inv(extex+lammyI)
    omegapre = np.matmul(extexinv, eX.T)
    return np.matmul(omegapre, y)


def scramble(labels,samples):
    # shuffling of data points
    labels = np.reshape(labels,(506,1))
    adjoined = np.concatenate((samples,labels),axis=1)
    np.random.shuffle(adjoined)
    samples = adjoined.T[0:13].T
    labels = adjoined[:,13]
    return [labels,samples]

def crossValidationRR(labels,samples):
    best_error_array = [] # the error associated with the best lambda
    postscrambled = scramble(labels,samples)
    labels = postscrambled[0]
    samples = postscrambled[1]
    
    error = 100000 # initialized to arbitrary large number 
    best_lambda = 0 # initialized best lambda
    possible_lambda = np.arange(0,1,0.003) # testing all possible lambdas 
    for lammy in possible_lambda:
        error_array = [] # error array associated with lambda based on 3 train-1 validate 
        final_error_array = [] # error array associated with lambda based on 4 train-1 validate 
        for k in range(0,5):
            if k == 4:
                sindex1 = 404
                sindex2 = 506
                sondex1 = 0
                sondex2 = sondex1+101
            else:
                sindex1 = (k*101)
                sindex2 = sindex1+101
                sondex1 = ((k+1)*101)
                sondex2 = sondex1+101
            leave_set = samples[sindex1:sindex2]
            leave_labels = labels[sindex1:sindex2]
            validation_set = samples[sondex1:sondex2]
            validation_labels = labels[sondex1:sondex2]
            if k!= 4:
                training_set = np.concatenate((samples[sondex2:506],samples[0:sindex1]))
                training_labels = np.concatenate((labels[sondex2:506],labels[0:sindex1]))
            else:
                training_set = samples[sondex2:sindex1]
                training_labels = labels[sondex2:sindex1]
                
            # cross validating to find optimal lambda
            omega = rrLS(training_labels,training_set,lammy)
            augmented_test = augment(validation_set)
            predicted_out = np.matmul(augmented_test, omega)
            tlabs = np.reshape(validation_labels,(predicted_out.shape[0],1))
            diff = np.linalg.norm(predicted_out-tlabs,2)
            error_array.append((diff**2)/predicted_out.shape[0])
            
            # since i have my lambda, i test its performance on the leave set
            newtraining_set = np.concatenate((validation_set,training_set))
            newtraining_labels = np.concatenate((validation_labels,training_labels))
            omega = rrLS(newtraining_labels,newtraining_set,lammy)
            augmented_test = augment(leave_set)
            predicted_out = np.matmul(augmented_test,omega)
            tlabs = np.reshape(leave_labels,(predicted_out.shape[0],1))
            diff = np.linalg.norm(predicted_out-tlabs,2)
            final_error_array.append((diff**2)/predicted_out.shape[0])
                                             
        
        if(mean(error_array) < error): # see which labmda gives me the smallest error
                error = mean(error_array)
                best_lambda = lammy
                best_error_array = final_error_array # error array associated with the best lambda on 4train, 1 test
                
                
    return [best_lambda,best_error_array]


out = crossValidationRR(labels,samples)
print("Best lambda:",out[0])
print("This lambda's performance (error array)", out[1])
print("Average prediction error", mean(out[1]))

NameError: name 'labels' is not defined